 que obtengamos el número de estaciones normales recorridas y los trasbordos. Lo que queremos es lo siguiente: <br>
* Tomamos aletoriamente dos estaciones $inicio = rand(0, N)$ $ fin = rand(0, N)$, pedimos $inicio != fin$ con un if.
* Calculamos el número de estaciones normale y el número de trasbordos y después hacemos lo siguiente
* Cuando un pasajero se sube al metro espera un tiempo $t_{espera}$ a que pase el metro, suponemos que máximo el metro tarda en pasar $t_{max}$ (ej. a lo más tarda 15 minutos, entonces $t_{max} = 15\times60 s$). Suponemos que la distribución es uniforme (aunque después sería bueno asumir distribución gaussiana o poisson... pero después vemos eso).
Entonces $t_{espera} = rand()*t_{max}$. <br>
* Cuando el metro se detiene en una estación se detiene a lo menos $t_{parada_min}$ y a lo más $t_{parada_max}$. Entonces por cada estación que pare el metro se debe de tomar en cuenta $t_{parada} = rand(t_{parada_min}, t_{parada_max})$ (o como sea que se ponga en la función) <br>
* Cuando el pasajero hace un trasbordo entonces tarda en caminar un tiempo $t_{trasbordo}$ y además debe de esperar a que vuelva a pasar el camión. Volvemos a calcular un $t_{espera} = rand()*t_{max}$.
* Finalmente sumamos todo el tiempo y vemos total de estaciones que recorrio y lo guardamos en una lista apropiada. Después vamos con el siguiente pasajero y hacemos lo mismo.

In [3]:
tprom_espera = 4*60. #En promedio esperas 4 min a que pase el metro
t_viaje = 1*60 #El metro tarda 1 minuto en pasar de una estación a otra
tprom_parada = 20. #El metro se para 20 segundos en promedio en cada estación
t_tras = 1*60. #Tardas 1 minuto en hacer el trasbordo (parte a pie)
normal_edge = t_viaje + tprom_parada
transit_edge = t_tras + tprom_espera + 0.1 #los puntos flotantes nos van a ayudar a distinguir
express_edge = 2*t_viaje + tprom_parada + 0.001


140.001

In [9]:
a = 140.001
round(a%1, 10)*1000 #para obtener estaciones express

1.0

In [10]:
function NormalesTrasbordosyExpress(D_normal, D_express, i, j, normal_edge, transit_edge, express_edge)
    num_tras_norm = round(D_normal[i, j]%1, 1)*10
    num_est = round((D_normal[i, j] - transit_edge*num_tras)/normal_edge, 1)
    num_tras_exp = round(D_express[i, j]%1, 1)*10
    num_exp = round(D_express[i, j]%1, 10)*1000
    return num_tras_norm, num_est, num_tras_exp, num_exp
end

NormalesTrasbordosyExpress (generic function with 1 method)

In [11]:
function PasajerosGaussiana(D_normal, D_express, tprom_espera, t_viaje, tprom_parada, t_tras, N_pasajeros, normal_edge, transit_edge, express_edge)
    
    N_est = length(DistMatrix[1, :])
    Stations_traveled  = zeros(N_pasajeros) #Esta es la misma para ambas
    Time_traveled_norm = zeros(N_pasajeros)
    Time_traveled_exp = zeros(N_pasajeros)
    Velocidad_norm = zeros(N_pasajeros)
    Velocidad_exp = zeros(N_pasajeros)
    
    for pasajero = 1:N_pasajeros
        initial_station = rand(1:N_est)
        end_station = rand(1:N_est)
        while GetStationName(initial_station) == GetStationName(end_station)
            initial_station = rand(1:N_est)
            end_station = rand(1:N_est) 
        end
        
        num_tras_norm, num_est, num_tras_exp, num_exp = NormalesTrasbordosyExpress(D_normal, D_express, initial_station, end_station, normal_edge, transit_edge, express_edge)
        
        #Parte del Metro normal
        tiempo_tras_norm = 0
        for tras = 1:num_tras_norm
            tiempo_tras_norm += tprom_espera*rand() + t_tras
        end
        tiempo_paradas_norm = 0
        for parada = 1:num_est
            tiempo_paradas_norm += tprom_parada*rand() + t_viaje
        end
        tiempito_norm = tprom_espera*rand() + tiempo_tras_norm + tiempo_paradas_norm
        Time_traveled_norm[pasajero] = tiempito_norm
        Velocidad_norm[pasajero] = num_est/tiempito_norm
        
        #Parte del Metro Express
        tiempo_tras_exp = 0
        for tras = 1:num_tras_exp
            tiempo_tras_exp += tprom_espera*rand() + t_tras
        end
        tiempo_paradas_exp = 0
        for parada = 1:num_exp
            tiempo_paradas_exp += tprom_parada*rand() + 2*t_viaje #Pues el express tarda el doble en un viaje
        end
        tiempito_exp = tprom_espera*rand() + tiempo_tras_exp + tiempo_paradas_exp
        Time_traveled_norm[pasajero] = tiempito_exp
        Velocidad_norm[pasajero] = num_est/tiempito_exp
        
    end
    return Stations_traveled, Time_traveled_norm, Velocidad_norm, Time_traveled_exp, Velocidad_exp
end

PasajerosGaussiana (generic function with 1 method)

In [3]:
Estaciones, Tiempo, Vel = PasajerosMonteCarlo(D, tprom_espera, t_viaje, tprom_parada, t_tras, 1000000, transit_edge_dist, normal_edge_dist)

LoadError: PasajerosMonteCarlo not defined
while loading In[3], in expression starting on line 1

In [4]:
plot(Estaciones, Tiempo, ".")
xlabel("Estaciones recorridas por el pasajero")
ylabel("Tiempo de Recorrido")

LoadError: plot not defined
while loading In[4], in expression starting on line 1

In [ ]:
function PasajerosMonteCarlo2(DistMatrix, tprom_espera, t_viaje, tprom_parada, t_tras, N_pasajeros, transit_edge_dist, normal_edge_dist)
    N_est = length(DistMatrix[1, :])
    Stations_traveled = zeros(N_pasajeros)
    Time_traveled = zeros(N_pasajeros)
    Velocidad = zeros(N_pasajeros)
    for pasajero = 1:N_pasajeros
        initial_station = rand(1:N_est)
        end_station = rand(1:N_est)
        while GetStationName(initial_station) == GetStationName(end_station)
            initial_station = rand(1:N_est)
            end_station = rand(1:N_est) 
        end
        num_est, num_tras = NormalesyTrasbordos(D, initial_station, end_station, transit_edge_dist, normal_edge_dist)
        Stations_traveled[pasajero] = num_est
        tiempo_tras = 0
        for tras = 1:num_tras
            tiempo_tras += tprom_espera*rand(Poisson()) + t_tras
        end
        tiempo_paradas = 0
        for parada = 1:num_est
            tiempo_paradas += tprom_parada*rand(Poisson()) + t_viaje
        end
        tiempito = tprom_espera*rand(Poisson()) + tiempo_tras + tiempo_paradas
        Time_traveled[pasajero] = tiempito
        Velocidad[pasajero] = num_est/tiempito
        
    end
    return Stations_traveled, Time_traveled, Velocidad
end